In [16]:
import torch
import torch.nn.functional as F
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
# taken from end makemore2. most values updated to match K's

# no. of chars used in context
block_size = 3

# dimensionality of embedding table
n_embd = 10
n_weights = block_size * n_embd

# no. of neurons in hidden layer
n_hidden = 200

# size of batches
batch_size = 32

# number of training runs
train_runs = 20001

# learning rate
l_rate_init = 0.1

#     learning rate decay
decay_factor = 0.99999

# others
# ???

In [19]:
kaiming_init = (5/3) / (n_weights**0.5)

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), 	generator = g) 
W1 = torch.randn((n_weights, n_hidden), 	generator = g) * kaiming_init
b1 = torch.randn(n_hidden, 					generator = g) * 0.1 # actual useless due to layer norming
# NB - DO NOT set W2 to zeroes (K asserts at this point in vid, does not yet explain)
W2 = torch.randn((n_hidden, vocab_size), 	generator = g) * 0.1 
b2 = torch.randn(vocab_size, 				generator = g) * 0.1

bn_gain = torch.ones((1, n_hidden)) * 0.1 + 1.0
bn_bias = torch.zeros((1, n_hidden)) * 0.1

parameters = [C, W1, b1, W2, b2, bn_gain, bn_bias]

for p in parameters:
    p.requires_grad = True 
    
step_i = []
loss_i = []
log_loss_i = []

print(f'No. parameters: {sum(p.nelement() for p in parameters)}')

No. parameters: 12297


In [20]:
# get all words
words = open('names.txt', 'r').read().splitlines()


# get indexes between chars and ints
chars = sorted(list(set(''.join(words))))
s_to_i = {s:i+1 for i, s in enumerate(chars)}
s_to_i['.'] = 0
i_to_s = {i:s for s, i in s_to_i.items()}
vocab_size = len(s_to_i)
print(f'First 15 words: {words[:15]}\n\ns to i: {s_to_i}\n\ni to s: {i_to_s}\n\nVocab size: {vocab_size}')

First 15 words: ['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn', 'abigail', 'emily', 'elizabeth', 'mila', 'ella']

s to i: {'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}

i to s: {1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}

Vocab size: 27


In [21]:
def build_dataset(words, block_size):
    block_size = block_size
    X, Y = [], []
    for word in words:
        # print(word)
        context = [0] * block_size
    
        for char in word + '.':
            ix = s_to_i[char]
            X.append(context)
            Y.append(ix)
            # print(''.join(i_to_s[i] for i in context), '->', i_to_s[ix])
            context = context[1:] + [ix] # crop first character and append next
    
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    
    print(f'X shape: {X.shape} \tX dtype: {X.dtype} \tY shape: {Y.shape} \tY dtype: {Y.dtype}')
    return X, Y

In [22]:
# set train, dev, test sets

random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

X_trn, Y_trn = build_dataset(words[:n1], block_size)
X_dev, Y_dev = build_dataset(words[n1:n2], block_size)
X_tst, Y_tst = build_dataset(words[n2:], block_size)

X shape: torch.Size([182606, 3]) 	X dtype: torch.int64 	Y shape: torch.Size([182606]) 	Y dtype: torch.int64
X shape: torch.Size([22697, 3]) 	X dtype: torch.int64 	Y shape: torch.Size([22697]) 	Y dtype: torch.int64
X shape: torch.Size([22844, 3]) 	X dtype: torch.int64 	Y shape: torch.Size([22844]) 	Y dtype: torch.int64


In [23]:
# function to be used in comparing manual and pytorch gradients

def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()        # check if values match exactly
    app = torch.allclose(dt, t.grad)           # check for v closeness e.g. stemming from float calc diffs
    maxdiff = (dt - t.grad).abs().max().item() # return max diff between manual and pytorch gradients
    print(f'{s:15s} \tExact: {str(ex):5s} \tApprox: {str(app):5s} \tmaxDiff: {maxdiff}')

In [24]:
# make batch for X and Y
ix = torch.randint(0, X_trn.shape[0], (batch_size,), generator = g)
Xb, Yb = X_trn[ix], Y_trn[ix]

In [25]:
Yb

tensor([ 8, 14,  5, 14,  0,  9,  1, 12,  0,  4, 22, 18, 25, 14,  9,  0,  5,  1,
         5,  5, 21,  5,  9,  0,  1, 14, 25,  9,  0,  1,  8,  5])

In [ ]:
 # forward pass broken into smaller steps to allow backprop one element by one

emb = C[Xb] # embed characters into vectors
emb_cat = emb.view(emb.shape[0], -1) # concatenate vectors

# linear layer 1
hl_pre_bn = emb_cat @ W1 + b1 # hidden layer pre-activation

# batch norm layer
bn_mean_i = 1 / batch_size * hl_pre_bn.sum(0, keepdim = True)
bn_diff = hl_pre_bn - bn_mean_i
bn_diff_sq = bn_diff ** 2
bn_var = 1 / (batch_size - 1) * (bn_diff_sq).sum(0, keepdim = True) # Bessel's correction
bn_var_inv = (bn_var + 1e-5) ** -0.5

In [ ]:
# backprop through all above variables one by one

# create array of zeroes in shape of logprobs
d_logprobs = torch.zeros_like(logprobs)
d_logprobs[range(n), Yb] = -1.0 / batch_size
